<a href="https://colab.research.google.com/github/MuleHakim/Pytorch-Tutorial/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
# Conver MNIST Image files into a Tensor of 4D (# no of images, height, width, color channel)
transform = transforms.ToTensor()

In [23]:
# Train data
train_data = datasets.MNIST(root='./cnn_data', train=True, transform=transform, download=True)

In [24]:
# Test data
test_data = datasets.MNIST(root='./cnn_data', train=False, transform=transform, download=True)

In [25]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [26]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [27]:
# Create a small batch size for images
train_loader = DataLoader(train_data,batch_size=10,shuffle=True)
test_loader = DataLoader(test_data,batch_size=10,shuffle=False)

In [28]:
# Define our CNN model
# Describe convolutional layer
conv1 = nn.Conv2d(1,6,3,1)
conv2 = nn.Conv2d(6,16,3,1)

In [29]:
# Grab 1 MNIST record/image
for i, (X_train,y_train) in enumerate(train_data):
  break

In [30]:
X_train

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

In [31]:
X_train.shape

torch.Size([1, 28, 28])

In [32]:
x = X_train.view(1,1,28,28)

In [33]:
# Perform our first convolution
x = F.relu(conv1(x)) # Rectified Linear Unit for our activation function

In [34]:
# 1 single image, 6 is the filters, 26x26
x.shape # We didn't set padding so we lose 2 pixels around outside of the image

torch.Size([1, 6, 26, 26])

In [35]:
# pass thru the pooling layer
x = F.max_pool2d(x,2,2)   # kernel of 2 and stride of 2

In [36]:
x.shape  # 26 / 2 =13

torch.Size([1, 6, 13, 13])

In [37]:
# Do our second convolutional layer
x = F.relu(conv2(x))

In [38]:
x.shape # Again we didn't set padding so we lose 2 pixels around outside of the image

torch.Size([1, 16, 11, 11])

In [39]:
# Pooling layer
x = F.max_pool2d(x,2,2)   # kernel of 2 and stride of 2

In [40]:
x.shape    # 11 / 2 = 5.5 but we have to round down, because you can't invent data to round up

torch.Size([1, 16, 5, 5])

In [42]:
# Model class
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1,6,3,1)
    self.conv2 = nn.Conv2d(6,16,3,1)
    # Fully connected layer
    self.fc1 = nn.Linear(5*5*16,120)
    self.fc2 = nn.Linear(120,70)
    self.fc3 = nn.Linear(70,10)

  def forwaed(self,X):
    # First pass
    X = F.relu(self.conv1(X))
    X = F.max_pool2d(X,2,2) # 2x2 kernel and stride of 2
    # Second pass
    X = F.relu(self.conv2(X))
    X = F.max_pool2d(X,2,2) # 2x2 kernel and stride of 2

    # Review to flatten it out
    X = X.view(-1,16*5*5)  # -1 so that we can  vary the batch size

    # Fully connected layers
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.fc3(X)

    return F.log_softmax(X,dim=1)

In [43]:
# Create an instance of our model
torch.manual_seed(97)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=70, bias=True)
  (fc3): Linear(in_features=70, out_features=10, bias=True)
)

In [44]:
# Loss function optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)  # smaller the lr, longer its gonna take to train